In [1]:
import re
import os
import sys
import collections
import pandas as pd
import numpy as np

from keras.optimizers import *
from keras.callbacks import *
from keras.models import *
from keras.layers import *
from keras.initializers import *
from keras.activations import *
from keras_layer_normalization import LayerNormalization

import tensorflow as tf
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')

data_en_path = "data/en/train"
data_en_10k_path = "data/en-10k/"
data_en_valid_path = "data/en-valid/"
data_en_valid_10k_path = "data/en-valid-10k/"

raw_task_category = [
    'single-supporting-fact'
    ,'two-supporting-facts'
    ,'three-supporting-facts'
    ,'two-arg-relations'
    ,'three-arg-relations'
    ,'yes-no-questions'
    ,'counting'
    ,'lists-sets'
    ,'simple-negation'
    ,'indefinite-knowledge'
    ,'basic-coreference'
    ,'conjunction'
    ,'compound-coreference'
    ,'time-reasoning'
    ,'basic-deduction'
    ,'basic-induction'
    ,'positional-reasoning'
    ,'size-reasoning'
    ,'path-finding'
    ,'agents-motivations'
]

tasks = [
    'single-supporting-fact'
#     ,'two-supporting-facts'
#     ,'three-supporting-facts'
#     ,'two-arg-relations'
#     ,'three-arg-relations'
#     ,'yes-no-questions'
#     ,'counting'
#     ,'lists-sets'
#     ,'simple-negation'
#     ,'indefinite-knowledge'
#     ,'basic-coreference'
#     ,'conjunction'
#     ,'compound-coreference'
#     ,'time-reasoning'
#     ,'basic-deduction'
#     ,'basic-induction'
#     ,'positional-reasoning'
#     ,'size-reasoning'
#     ,'path-finding'
#     ,'agents-motivations'
]

dropout = 0.3

Using TensorFlow backend.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ICPS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
class Sentence:
    def __init__(self, num, sentence):
        self.num = num
        self.sentence = sentence

class bAbISet:
    def __init__(self, task=None, sentences=None, question=None, answer=None, supporting_num=None):
        self.task = task
        self.sentences = sentences
        self.story_len = len(self.sentences)
        self.question = question
        self.answer = answer
        self.supporting_num = supporting_num
        
    def add_vec_data(self, vec_sentences, vec_question, vec_answer):
        self.vec_sentences = vec_sentences
        self.vec_question = vec_question
        self.vec_answer = vec_answer
        
    def Print(self):
        print("=============================================================")
        print(">> Task: ", self.task)
        print(">> Sentences: ", self.story_len)
        for sentence in self.sentences:
            print(sentence.num, ": ", sentence.sentence)
        print(">> Question: ", self.question)
        print(">> Answer: ", self.answer)
        print(">> Supporting Fact: ", self.supporting_num)
        print("=============================================================")
        
    def Print_vec(self):
        if self.vec_sentences is None:
            return
        
        print("=============================================================")
        print(">> Task: ", self.task)
        print(">> Sentences: ", len(self.sentences))
        for sentence in self.vec_sentences:
            print(sentence.num, ": ", sentence.sentence)
        print(">> Question: ", self.vec_question)
        print(">> Answer: ", self.vec_answer)
        print(">> Supporting Fact: ", self.supporting_num)
        print("=============================================================")
        

In [3]:
class bAbIUtils:
    def __init__(self, path):
        self.files = os.listdir(path)
        self.files = [os.path.join(path, f) for f in self.files]
        
    def make_question_set(self, sentences):
        all_set = []
        temp_set = []
        for sentence in sentences:
            temp_set.append(sentence.lower())
            if sentence.find('\t') != -1:
                all_set.append(temp_set.copy())
                temp_set.clear()
                
        return all_set
                
    def data_processing(self):
        print("※ Data Processing...")
        all_data = []
        for file in self.files:
            
            idx_start = file.find('_') + 1
            idx_end = file[idx_start:].find('_')
            task = file[idx_start:idx_start + idx_end]
            
            if task not in tasks:
                continue
                
            print(task)
            
            with open(file, 'r') as f:
                data = f.readlines()
                raw_set = self.make_question_set(data)
                
                for one_set in raw_set:
                    Sentence_set = []
                    for sentence in one_set:
                        sentence = sentence.replace('\n', '')
                        
                        idx = sentence.find(' ')
                        idx_answer =  sentence.find('\t')
                        if (len(Sentence_set) == 0) & (sentence[:idx] != '1'):
                            Sentence_set.extend(temp)
                        
                        if idx_answer != -1: # Answer Detected
                            question = sentence[idx + 1:idx_answer]
                            idx_answer_end = sentence[idx_answer + 1:].find('\t')
                            answer = sentence[idx_answer + 1: idx_answer + idx_answer_end + 1]
                            supporting_num = sentence[idx_answer + idx_answer_end + 2:]
                            
                        else:
                            Sentence_set.append(Sentence(sentence[:idx], sentence[idx + 1:]))
                    
                        temp = Sentence_set.copy()
                            
                    all_data.append(bAbISet(task, Sentence_set, question, answer, supporting_num))
                    
        return all_data
    
    def make_dictionary(self, all_set):
        print(">> Make Dictionary...")
        
        sentence_set = []
        for one_set in all_set:
            for Sentence in one_set.sentences:
                sentence_set.append(Sentence.sentence)
            sentence_set.append(one_set.question)
            sentence_set.append(one_set.answer)
        
        words = []
        for sentence in sentence_set:
            tokens = nltk.word_tokenize(sentence)
                        
            words.extend(tokens)

        words = collections.Counter(words)

        dictionary = {}
        dictionary['<PAD>'] = 0
        dictionary['<UNK>'] = 1
        dictionary['<EOS>'] = 2
        dictionary['<S>'] = 3
        idx = 4
        for word in words.most_common():
            if len(word[0]) > 0:
                dictionary[word[0]] = idx
                idx += 1
            
            if idx >= 20000: break;

        return dictionary
    
    def vectorize_sentence(self, sentence, dictionary):
        vec_sentence = []
        #vec_sentence.append(dictionary['<S>'])
        for word in nltk.word_tokenize(sentence):
            if len(word) > 0:
                if word in dictionary:
                    vec_sentence.append(dictionary[word])
                else:
                    vec_sentence.append(dictionary['<UNK>'])
        
#         if add_padding == True:
#             for _ in range(len(vec_sentence), bAbI_max_len):
#                 vec_sentence.append(dictionary['<PAD>'])
        #vec_sentence.append(dictionary['<EOS>'])
        
        return np.array(vec_sentence)
    
    def add_padding(self, sentence, max_len):
        for _ in range(len(sentence), max_len):
            sentence = np.append(sentence, dictionary['<PAD>'])
        
        return sentence
        

In [4]:
bAbI_util = bAbIUtils(data_en_10k_path)
bAbI = bAbI_util.data_processing()
dictionary = bAbI_util.make_dictionary(bAbI)

print("Dictionary Size: ", len(dictionary))
print("Done")

※ Data Processing...
single-supporting-fact
single-supporting-fact
>> Make Dictionary...
Dictionary Size:  25
Done


In [5]:
max_sentences_len = 0
max_question_len = 0
max_answer_len = 1
max_story_size = 0
count = 0
print("Step 1")
for one_set in bAbI:
    count = count + 1
    print("\rSet : {0} / {1}".format(count, len(bAbI)), end='')
    if max_story_size < one_set.story_len:
        max_story_size = one_set.story_len
    
    vec_sentences = []
    for sentence in one_set.sentences:
        vec_sentences.extend(bAbI_util.vectorize_sentence(sentence.sentence, dictionary))
#         vec_sentence = bAbI_util.vectorize_sentence(sentence.sentence, dictionary)
#         vec_sentences.append(Sentence(sentence.num, vec_sentence))

#         if max_len < len(vec_sentence):
#             max_len = len(vec_sentence)
        if max_sentences_len < len(vec_sentences):
            max_sentences_len = len(vec_sentences)
        
    vec_question = bAbI_util.vectorize_sentence(one_set.question, dictionary)
    vec_answer = bAbI_util.vectorize_sentence(one_set.answer, dictionary)
    one_set.add_vec_data(vec_sentences, vec_question, vec_answer)
    
    if len(vec_answer) > 1:
        one_set.Print()
        
    if max_question_len < len(vec_question):
        max_question_len = len(vec_question)
    
count = 0
print("\nStep 2")
for one_set in bAbI:
    count = count + 1
    print("\rSet : {0} / {1}".format(count, len(bAbI)), end='')
    
#     for i in range(max_story_size):
#         if i < len(one_set.vec_sentences):
#             one_set.vec_sentences[i].sentence = bAbI_util.add_padding(one_set.vec_sentences[i].sentence, max_len)
#         else:
#             one_set.vec_sentences.append(Sentence(0, bAbI_util.add_padding([], max_len)))

    one_set.vec_sentences = bAbI_util.add_padding(one_set.vec_sentences, max_sentences_len)    
    one_set.vec_question = bAbI_util.add_padding(one_set.vec_question, max_question_len)

print("\nDone")

Step 1
Set : 11000 / 11000
Step 2
Set : 11000 / 11000
Done


In [6]:
# MAX_DICT_SIZE = len(dictionary)
# MAX_SEN_LEN = max_len
# MAX_STORY_SIZE = max_story_size
# MAX_ANSWER_LEN = max_answer_len
# D_MODEL = 64

# print("MAX_DICT_SIZE = ", MAX_DICT_SIZE)
# print("MAX_STORY_SIZE = ", MAX_STORY_SIZE)
# print("MAX_SEN_LEN = ", MAX_SEN_LEN)
# print("MAX_ANSWER_LEN = ", MAX_ANSWER_LEN)

# story_train = np.zeros((len(bAbI), MAX_STORY_SIZE, MAX_SEN_LEN), dtype=int)
# question_train = np.zeros((len(bAbI), MAX_SEN_LEN), dtype=int)
# answer_train = np.zeros((len(bAbI), MAX_ANSWER_LEN), dtype=int)

# count = 0
# for idx, one_set in enumerate(bAbI):
#     count = count + 1
#     print("\rSet : {0} / {1}".format(idx + 1, len(bAbI)), end='')
    
#     for i, sentence in enumerate(one_set.vec_sentences):
#         story_train[idx][i] = sentence.sentence
        
#     question_train[idx] = one_set.vec_question.copy()
#     answer_train[idx] = one_set.vec_answer.copy()
    
# story_train = np.array(story_train)
# question_train = np.array(question_train)
# answer_train = np.array(answer_train)

# print("\nstory_train = ", story_train.shape)
# print("question_train = ", question_train.shape)
# print("answer_train = ", answer_train.shape)

In [7]:
MAX_DICT_SIZE = len(dictionary)
MAX_SEN_LEN = max_sentences_len
MAX_Q_LEN = max_question_len
MAX_STORY_SIZE = max_story_size
MAX_ANSWER_LEN = max_answer_len
D_MODEL = 64

print("MAX_DICT_SIZE = ", MAX_DICT_SIZE)
print("MAX_STORY_SIZE = ", MAX_STORY_SIZE)
print("MAX_SEN_LEN = ", MAX_SEN_LEN)
print("MAX_Q_LEN = ", MAX_Q_LEN)
print("MAX_ANSWER_LEN = ", MAX_ANSWER_LEN)

story_train = np.zeros((len(bAbI), MAX_SEN_LEN), dtype=int)
question_train = np.zeros((len(bAbI), MAX_Q_LEN), dtype=int)
answer_train = np.zeros((len(bAbI), MAX_ANSWER_LEN), dtype=int)

count = 0
for idx, one_set in enumerate(bAbI):
    count = count + 1
    print("\rSet : {0} / {1}".format(idx + 1, len(bAbI)), end='')
    
#     for i, sentence in enumerate(one_set.vec_sentences):
#         story_train[idx][i] = sentence.sentence
        
    story_train[idx] = one_set.vec_sentences.copy()
    question_train[idx] = one_set.vec_question.copy()
    answer_train[idx] = one_set.vec_answer.copy()
    
story_train = np.array(story_train)
question_train = np.array(question_train)
answer_train = np.array(answer_train)

print("\nstory_train = ", story_train.shape)
print("question_train = ", question_train.shape)
print("answer_train = ", answer_train.shape)

MAX_DICT_SIZE =  25
MAX_STORY_SIZE =  10
MAX_SEN_LEN =  68
MAX_Q_LEN =  4
MAX_ANSWER_LEN =  1


Set : 1905 / 11000

Set : 2404 / 11000

Set : 3404 / 11000

Set : 3903 / 11000

...

Set : 8299 / 11000

Set : 8900 / 11000

Set : 9900 / 11000

Set : 10483 / 11000

Set : 11000 / 11000
story_train =  (11000, 68)
question_train =  (11000, 4)
answer_train =  (11000, 1)


In [10]:
class MEMN2N:
    def __init__(self):
        self.A = Sequential()
        self.A.add(Embedding(MAX_SEN_LEN, D_MODEL))
        self.A.add(Dropout(dropout))
        
        self.B = Sequential()
        self.B.add(Embedding(MAX_SEN_LEN, D_MODEL))
        self.B.add(Dropout(dropout))
        
        self.C = Sequential()
        self.C.add(Embedding(MAX_SEN_LEN, MAX_Q_LEN))
        self.C.add(Dropout(dropout))
        
    def compile(self, optimizer='adam'):
        story_input = Input(shape=(MAX_SEN_LEN, ), dtype='int32')
        question_input = Input(shape=(MAX_Q_LEN,), dtype='int32')
        
        story_encoded_m = self.A(story_input)
        story_encoded_c = self.C(story_input)
        u = self.B(question_input)
        
        #story_encoded_m = Lambda(lambda x:tf.reduce_sum(x, axis=2))(story_encoded_m)
        #story_encoded_c = Lambda(lambda x:tf.reduce_sum(x, axis=2))(story_encoded_c)
        #u = Lambda(lambda x:tf.reduce_sum(x, axis=1))(u)
        
        m = dot([story_encoded_m, u], axes=(2, 2))
        p = Activation('softmax')(m)
        
        c = add([p, story_encoded_c])
        o = Permute((2, 1))(c) 
        #o = Lambda(lambda x:tf.matmul(x[0], x[1]))([c, p])

        answer = concatenate([o, u])
        answer = LSTM(32)(answer)  # (samples, 32)
        answer = Dropout(dropout)(answer)  # (samples, 32)
        answer = Dense(MAX_DICT_SIZE, use_bias=False)(answer)
        answer = Activation('softmax')(answer)
        #answer = Lambda(lambda x:tf.argmax(x, axis=-1))(answer)
        #answer = Lambda(lambda x:tf.cast(x, 'int32'))(answer)
        
        self.model = Model([story_input, question_input], answer)
        self.model.compile(optimizer=optimizer,
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])


In [11]:
memn2n = MEMN2N()
memn2n.compile(Adam(0.001, 0.9, 0.98, epsilon=1e-9))
memn2n.model.fit([story_train, question_train], answer_train,
                 batch_size=32, epochs=100, validation_split=0.1)

Train on 9900 samples, validate on 1100 samples
Epoch 1/100
9900/9900 [==============================] - 5s 531us/step - loss: 1.9514 - acc: 0.1620 - val_loss: 1.8038 - val_acc: 0.1636
Epoch 2/100
9900/9900 [==============================] - 4s 387us/step - loss: 1.8187 - acc: 0.1658 - val_loss: 1.8045 - val_acc: 0.1664
Epoch 3/100
9900/9900 [==============================] - 4s 389us/step - loss: 1.8028 - acc: 0.1656 - val_loss: 1.7923 - val_acc: 0.1745
Epoch 4/100
9900/9900 [==============================] - 4s 389us/step - loss: 1.7901 - acc: 0.1833 - val_loss: 1.7771 - val_acc: 0.2482
Epoch 5/100
9900/9900 [==============================] - 4s 393us/step - loss: 1.7597 - acc: 0.2279 - val_loss: 1.7309 - val_acc: 0.2655
Epoch 6/100
9900/9900 [==============================] - 4s 387us/step - loss: 1.7218 - acc: 0.2590 - val_loss: 1.6900 - val_acc: 0.2700
Epoch 7/100
9900/9900 [==============================] - 4s 395us/step - loss: 1.6841 - acc: 0.2866 - val_loss: 1.6380 - val_acc: 

KeyboardInterrupt: 